In [18]:
import pandas as pd
df=pd.read_csv("Merge_2.csv")
df['거래일자'] = pd.to_datetime(df['거래일자']) # int -> datetime
df['거래취소여부'] = df['거래취소여부'].astype('int64') # float64 -> int64
df['판매금액'] = df['판매금액'].astype('int64') #float64 -> int64
df['서비스가격'] = df['서비스가격'].astype('int64') #float64 -> int64
df['이용자수'] = df['이용자수'].astype('int64') #float64 -> int64
df['서비스번호'] = df['서비스번호'].astype('int64') #float64 -> int64
df['총금액'] = df['총금액'].astype('int64') #float64 -> int64
df['매출'] = df['매출'].astype('int64') #float64 -> int64
df['연령'] = df['연령'].astype('int64') #float64 -> int64
df['총구매금액'] = df['총구매금액'].astype('int64') #float64 -> int64
df['총 판매 금액'] = df['총 판매 금액'].astype('int64') #float64 -> int64

In [19]:
df1=df.copy()
# 새로운 컬럼 '추가결제금액발생여부' 생성
df1['추가결제금액발생여부'] = df1['추가결제금액'].apply(lambda x: 0 if x == 0 else 1)
df1['추가결제금액발생여부'] = df1['추가결제금액발생여부'].astype('category')
df1['추가결제금액발생여부'].unique()

[1, 0]
Categories (2, int64): [0, 1]

서비스 가격 , 추가 결제금액 발생여부=> 거래취소율에 영향 

In [20]:
df_selected = df1[['서비스가격', '서비스번호', '거래취소여부']]

df_grouped = df_selected.groupby('서비스번호').agg(
    취소된_거래수=('거래취소여부', lambda x: (x == 1).sum()),  # 거래취소여부가 1인 취소된 거래 수
    전체_거래수=('거래취소여부', 'count')                    # 서비스명별 전체 거래 수
).reset_index()

# 거래취소율 계산
df_grouped['거래취소율'] = (df_grouped['취소된_거래수'] / df_grouped['전체_거래수']) * 100

# 서비스명별 가격은 첫 번째 값만 가져오기 (서비스번호별 가격은 동일하므로 첫 번째 값만 사용)
df_grouped['서비스가격'] = df.groupby('서비스번호')['서비스가격'].first().values

df_3 = df_grouped[['서비스가격', '서비스번호', '거래취소율']]

df_3

,서비스가격,서비스번호,거래취소율
0,5000,2102,1.259446
1,10000,2103,2.435897
2,50000,2105,2.087683
3,5000,2106,3.602058
4,5000,2107,1.207938
...,...,...,...
2857,850000,6488,0.000000
2858,550000,6489,0.000000
2859,380000,6494,0.000000
2860,33000,6495,0.000000


In [21]:
df2 = df1[['서비스번호','추가결제금액발생여부']]

In [22]:
df_4 = pd.merge(df_3,df2,on='서비스번호')

In [23]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335450 entries, 0 to 335449
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   서비스가격       335450 non-null  int64   
 1   서비스번호       335450 non-null  int64   
 2   거래취소율       335450 non-null  float64 
 3   추가결제금액발생여부  335450 non-null  category
dtypes: category(1), float64(1), int64(2)
memory usage: 8.0 MB


In [24]:
df_5 = df_4.copy()
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df_5['추가결제금액발생여부'] = label_encoder.fit_transform(df_5['추가결제금액발생여부'])

C:\Users\woota\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\woota\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [25]:
from scipy.stats import pearsonr

variables = ['서비스가격', '추가결제금액발생여부']
for var in variables:
    corr, p = pearsonr(df_5[var], df_5['거래취소율'])
    print(f"{var} ↔ 거래취소율 | 상관계수: {corr:.3f} | p-value: {p:.5f}")

서비스가격 ↔ 거래취소율 | 상관계수: 0.001 | p-value: 0.45173
추가결제금액발생여부 ↔ 거래취소율 | 상관계수: -0.012 | p-value: 0.00000


t 검정

In [27]:
from scipy.stats import levene

# 추가결제금액발생여부 그룹 나누기
group_0 = df_5[df_5['추가결제금액발생여부'] == 0]['거래취소율']
group_1 = df_5[df_5['추가결제금액발생여부'] == 1]['거래취소율']

# Levene’s Test 수행
stat, p_value = levene(group_0, group_1)

# 결과 출력
print(f"Levene's Test: 통계량={stat:.3f}, p-value={p_value:.5f}")

# 해석
if p_value < 0.05:
    print("귀무가설 기각: 두 그룹의 분산이 다릅니다 (등분산성 없음).")
else:
    print("귀무가설 채택: 두 그룹의 분산이 같습니다 (등분산성 있음).")

Levene's Test: 통계량=24.939, p-value=0.00000
귀무가설 기각: 두 그룹의 분산이 다릅니다 (등분산성 없음).


In [28]:
from scipy import stats

# 추가결제금액발생여부가 0인 그룹과 1인 그룹의 거래취소율 데이터 추출
group_0 = df_5[df_5['추가결제금액발생여부'] == 0]['거래취소율']
group_1 = df_5[df_5['추가결제금액발생여부'] == 1]['거래취소율']

# Welch’s t-test 수행 (등분산 가정 X)
t_stat, p_value = stats.ttest_ind(group_0, group_1, equal_var=False)

# 결과 출력
print(f"Welch's t-test: t-통계량={t_stat:.4f}, p-값={p_value:.4f}")

# 해석
if p_value < 0.05:
    print("추가결제금액발생여부에 따른 거래취소율 차이가 통계적으로 유의미함.")
else:
    print("추가결제금액발생여부에 따른 거래취소율 차이가 통계적으로 유의미하지 않음.")

Welch's t-test: t-통계량=6.7341, p-값=0.0000
추가결제금액발생여부에 따른 거래취소율 차이가 통계적으로 유의미함.
